In [ ]:
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
from dowhen import do, when

In [ ]:
p={'WG':3,'PADDING':2,'DEEP_ETCH_EBL':1,'DEEP_ETCH_PL':1}

In [ ]:
xs = cross_section_with_sleeves(0.43,5,radius=5,radius_min=5,sleeve_layer='DEEP_ETCH_EBL')
xs_routing = cross_section_with_sleeves(0.43,5,radius=20, radius_min=15,sleeve_layer='DEEP_ETCH_EBL')

In [ ]:
@gf.cell
def ring_resonator_fill_middle(**kwargs):
    ring_resonator_component = ring_resonator(**kwargs)
    ring_resonator_component.extract(['DEEP_ETCH_EBL']).get_polygons(layers=['DEEP_ETCH_EBL'])
    region = gf.kdb.Region(ring_resonator_component.extract(['DEEP_ETCH_EBL']).get_polygons(layers=['DEEP_ETCH_EBL'])[gf.get_layer('DEEP_ETCH_EBL')]).hulls()
    c = gf.Component()
    c << ring_resonator_component
    c.add_polygon(region,layer=gf.get_layer('DEEP_ETCH_EBL'))
    c.ports = ring_resonator_component.ports
    return c

In [ ]:
doubly_clamped_beam_with_round_support(width=0.1, length=20, support_length=0.2, create_mask=True).show()

In [ ]:
resonator_spec = partial(ring_resonator_fill_middle, gap=0.25, radius=20, length_x=5,length_y=0,cross_section=xs,bend="bend_circular")
# @gf.cell
def cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.15, gap=0.1, resonator_spec=resonator_spec, resonator_position=1, waveguide_spacing=250, x_span_resonator=20):
    c = gf.Component()
    position_factor = beam_l
    cantilever = cantilever_beam(length=beam_l, width=beam_w, deep_etch_layer='DEEP_ETCH_EBL')
    resonator = resonator_spec(x_span=x_span_resonator)
    cantilever_ref = c << cantilever
    resonator_ref = c << resonator
    
    
    resonator_ref.connect('p1', cantilever_ref.ports['s1'].copy(trans=gf.kdb.Trans(x=(1-resonator_position)*position_factor*1000,y=-int(gap*1000))),allow_width_mismatch=True)
    
    o1_ring = resonator_ref['o1']
    o2_ring = resonator_ref['o2']
    c.add_port(
        name='optical_in',
        width=o1_ring.width,
        orientation=-90,
        center=(o1_ring.center[0] - (waveguide_spacing- x_span_resonator)/2, o1_ring.center[1] -50),
        layer=o1_ring.layer,
        port_type='optical'
    )
    c.add_port(
        name='optical_out',
        width=o2_ring.width,
        orientation=-90,
        center=(o2_ring.center[0] + (waveguide_spacing- x_span_resonator)/2, o2_ring.center[1] -50),
        layer=o2_ring.layer,
        port_type='optical'
    )
    gf.routing.route_single(
        c,
        port1=o1_ring,
        port2=c.ports['optical_in'].copy(post_trans=gf.kdb.Trans.R180),
        cross_section=xs_routing,
    )
    gf.routing.route_single(
        c,
        port1=o2_ring,
        port2=c.ports['optical_out'].copy(post_trans=gf.kdb.Trans.R180),
        cross_section=xs_routing,
    )
    
    c = merge_layers_with_priority(c,p)
    c.info['params'] = {
        'beam_length': beam_l,
        'beam_width': beam_w,
        'gap': gap,
        'resonator_position': resonator_position,
    }
    
    return c

cantilever_beam_with_ring_resonator(resonator_position=1, beam_w=0.2, beam_l=10).show()

In [ ]:
# @gf.cell
def cc_beam_with_ring_resonator(beam_l=20, beam_w=0.1, gap=0.1, resonator_spec=resonator_spec, resonator_position=1, waveguide_spacing=250, x_span_resonator=20):
    import inspect
    import textwrap
    f = cantilever_beam_with_ring_resonator
    def cc_beam_with_base(beam_w, beam_l):
        c = gf.Component()
        beam = doubly_clamped_beam_with_round_support(width=beam_w, length=beam_l, support_length=(beam_w+0.05,0.3))
        beam_ref = c << beam
        base = gf.Component()
        base << gf.components.rectangle(size=(5,5), layer="WG")
        
        base.ports = gf.components.rectangle(size=(5,5), layer  ="WG").ports
        base_ref1 = c << base
        base_ref2 = c << base
        base_ref1.connect('e1', beam_ref.ports['w1'].copy(gf.kdb.Trans(y=2500-(beam_w+0.05)/2*1000)), allow_width_mismatch=True, allow_type_mismatch=True)
        base_ref2.connect('e1', beam_ref.ports['e1'].copy(gf.kdb.Trans(y=2500-(beam_w+0.05)/2*1000)), allow_width_mismatch=True, allow_type_mismatch=True)
        c = merge_layers_with_priority(c,p)
        create_deep_etch_mask(c, 'bbox', mask_offset=5, deep_etch_layer='DEEP_ETCH_EBL')
        c.ports = beam_ref.ports
        return c
    
    
    with do(
        textwrap.dedent(inspect.getsource(cc_beam_with_base))
        ).when(
            f,"resonator = resonator_spec(x_span=x_span_resonator)"
            ).do(f"""
beam_l = {beam_l}
beam_w = {beam_w}
gap = {gap}
position_factor = beam_l/2
cantilever = cc_beam_with_base(beam_w=beam_w, beam_l=beam_l)"""
                ):
        return f(beam_l=beam_l, beam_w=beam_w, gap=gap, resonator_spec=resonator_spec, resonator_position=resonator_position, waveguide_spacing=waveguide_spacing, x_span_resonator=x_span_resonator)
    

cc_beam_with_ring_resonator(beam_w=0.1,resonator_position=0.7).show()
    

In [ ]:
cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.1)

In [ ]:
device_list = [
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cc_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
]
device_list2 = [
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.1),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
    cantilever_beam_with_ring_resonator(beam_l=20, beam_w=0.3),
]

In [ ]:
l_list = [5, 20]
w_list = [100, 200]
position = [0.5,0.2,1]
cantilever_list = {}
cc_list = {}
for l in l_list:
    w_p_comb = list(itertools.product(w_list, position))
    cantilever_list[l] = []
    cc_list[l] = []
    for w, pos in w_p_comb:
        cantilever_list[l].append(cantilever_beam_with_ring_resonator(beam_l=l, beam_w=w/1000, resonator_position=pos))
        cc_list[l].append(cc_beam_with_ring_resonator(beam_l=l, beam_w=w/1000, resonator_position=pos))
    cantilever_list[l].insert(0, cantilever_list[l][0])
    cc_list[l].insert(0, cc_list[l][0])

In [ ]:
import json
gc_spec = partial(gf.components.grating_coupler_elliptical,nclad=1,n_periods=60,fiber_angle=8)
gc_normal = partial(my_coupler,gc_spec,deep_layer="DEEP_ETCH_EBL")
def full_device_array_with_grating_couplers(device_list:list):
    def dict_to_pretty_str(d):
        return json.dumps(d, indent=2)
    def device_array():
        text_spec = partial(text_outline, size=10, outline_width=0.2, with_mask=False,layer='DEEP_ETCH_EBL')
        c = gf.Component()
        device_refs = [c << d for d in device_list]
        for i, dev in enumerate(device_refs):
            dev.movex(i*400)
            labelme(c, dev, dict_to_pretty_str(device_list[i].info['params']), text_spec=text_spec, anchor="center", position=lambda d: (d.ports['optical_in'].x+100,d.ymin))
            
        c.ports = {}
        for i, dev in enumerate(device_refs):
            c.add_ports([dev.ports["optical_in"], dev.ports["optical_out"]])
        c.auto_rename_ports()
        return c
    c = gf.Component()
    my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43,deep_layer="DEEP_ETCH_EBL")
    gc_array = gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing).copy()
    gc_array_ref = c << gc_array
    gc_array_ref.move((-1000,-500))
    device_array_ref = c << device_array()
    

    # align device array ports to grating coupler array ports
    device_array_ref.movex(
        origin=device_array_ref.ports['o8'].center[0],
        destination=gc_array_ref.ports[6].center[0]
    )
    device_array_ref_ports = [
        gf.Port(port.name, center=port.center, width=port.width, orientation=port.orientation, layer=port.layer)
        for port in device_array_ref.ports
    ]

    f = gf.routing.route_bundle(
        c,
        gc_array_ref.ports,
        device_array_ref_ports,
        cross_section=xs_routing,
        sort_ports=True,
        start_straight_length=200,
        separation=10
        
    )
    return c
c = full_device_array_with_grating_couplers(cantilever_list[5])
c.show()

In [ ]:
def full_device_array_with_grating_coupler_pair(device_list:list):
    def dict_to_pretty_str(d):
        return json.dumps(d, indent=2)
    def device_array():
        text_spec = partial(text_outline, size=10, outline_width=0.2, with_mask=False,layer='DEEP_ETCH_EBL')
        c = gf.Component()
        device_refs = [c << d for d in device_list]
        for i, dev in enumerate(device_refs):
            dev.movex(i*600)
            labelme(c, dev, dict_to_pretty_str(device_list[i].info['params']), text_spec=text_spec, anchor="center", position=lambda d: (d.ports['optical_in'].x+100,d.ymin))
            
        c.ports = {}
        for i, dev in enumerate(device_refs):
            c.add_ports([dev.ports["optical_in"], dev.ports["optical_out"]])
        c.auto_rename_ports()
        return c
    c = gf.Component()
    my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43,deep_layer="DEEP_ETCH_EBL")
    device_array_ref = c << device_array()
    device_array_ref_ports = [
        gf.Port(port.name, center=port.center, width=port.width, orientation=port.orientation, layer=port.layer)
        for port in device_array_ref.ports
    ]
    for i, port in enumerate(device_array_ref_ports):
        gc_ref = c << my_coupler_spec()
        if i % 2 == 0:
            gc_ref.connect(
            'o1',
            port.copy(trans=gf.kdb.Trans(x=-50 * 1000, y=-50 * 1000), post_trans=gf.kdb.Trans.R270),
            )
        else:
            gc_ref.connect(
            'o1',
            port.copy(trans=gf.kdb.Trans(x=50 * 1000, y=-50 * 1000), post_trans=gf.kdb.Trans.R90),
            )
        gf.routing.route_single(
            c,
            port1=gc_ref.ports['o1'],
            port2=port,
            cross_section=xs_routing,
        )
        
    return c
full_device_array_with_grating_coupler_pair(cantilever_list[5]).show()

In [ ]:
layout_final = gf.Component()
layout_final << die_with_alignment_marks(1.5e4,layers=['MARKER'])
layout_final << frame(size=19000, layers=["DEEP_ETCH_EBL","SHALLOW_ETCH"])

device_copy = gf.Component()
device_arrays_gc = []
device_arrays_gc_pair = []
lengths_gc = []
lengths_gc_pair = []
for l, devices in cantilever_list.items():
    device_array_gc_array = full_device_array_with_grating_couplers(devices)
    device_array_ref = device_copy << device_array_gc_array
    device_arrays_gc.append(device_array_ref)
    lengths_gc.append(l)

    
    device_array_gc_pair = full_device_array_with_grating_coupler_pair(devices)
    device_array_pair_ref = device_copy << device_array_gc_pair
    device_arrays_gc_pair.append(device_array_pair_ref)
    lengths_gc_pair.append(l)
    
for l, devices in cc_list.items():
    device_array_gc_array = full_device_array_with_grating_couplers(devices)
    device_array_ref = device_copy << device_array_gc_array
    device_arrays_gc.append(device_array_ref)
    lengths_gc.append(l)
    
    device_array_gc_pair = full_device_array_with_grating_coupler_pair(devices)
    device_array_pair_ref = device_copy << device_array_gc_pair
    device_arrays_gc_pair.append(device_array_pair_ref)
    lengths_gc_pair.append(l)
text_spec = partial(text_outline, size=50, outline_width=3, with_mask=False,layer='DEEP_ETCH_EBL')   
for i, device in enumerate(device_arrays_gc):
    device.move(origin=(device.xmin, device.ymax), destination=(-7000,6000-1300*i))
    labelme(device_copy, device, f"Length = {lengths_gc[i]} um", text_spec=text_spec, anchor="NE", position=lambda d: (d.xmax-50, d.ymin-50))
for i, device in enumerate(device_arrays_gc_pair):
    device.move(origin=(device.xmin, device.ymax), destination=(-7000,6300-1300*i))
    
    labelme(device_copy, device, f"Length = {lengths_gc_pair[i]} um", text_spec=text_spec, anchor="NE", position=lambda d: (d.xmax-50, d.ymin-50))
my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43,deep_layer="DEEP_ETCH_EBL")
gc_test = device_copy << gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing)
gc_test.movex(origin=gc_test.xmin, destination=device_copy.xmin)
gc_test.movey(800)
for i in range(0,14,2):
    gf.routing.route_single(
        device_copy,
        port1=gc_test.ports[f'o{i+1}'],
        port2=gc_test.ports[f'o{i+2}'],
        cross_section=xs_routing,
    )

(layout_final << device_copy).move((0,700))
(layout_final << device_copy).move((5000,700))
(layout_final << device_copy).move((10000,700))
layout_final.show()

In [ ]:
beam_gap_list = np.linspace(70,160,7)*1e-3
resonator_gap_list = np.array([150, 200, 250, 300])*1e-3
gap_sweep = gf.Component()
text_spec = partial(text_outline, size=50, outline_width=3, with_mask=False,layer='DEEP_ETCH_EBL')
for i, resonator_gap in enumerate(resonator_gap_list):
    resonator_spec = partial(ring_resonator_fill_middle, gap=resonator_gap, radius=20, length_x=5,length_y=0,cross_section=xs,bend="bend_circular")
    device_list_gap_sweep = [cantilever_beam_with_ring_resonator(beam_l=5, beam_w=0.1, gap=gap, resonator_spec=resonator_spec) for gap in beam_gap_list]
    gap_sweep_array = full_device_array_with_grating_couplers(device_list_gap_sweep)
    gap_sweep_array_pair = full_device_array_with_grating_coupler_pair(device_list_gap_sweep)
    gap_sweep_ref = gap_sweep << gap_sweep_array
    gap_sweep_ref.move((0,0-1300*i))
    labelme(gap_sweep, gap_sweep_ref, f"Resonator gap = {resonator_gap*1e3:.0f} nm", text_spec=text_spec, anchor="NE", position=lambda d: (d.xmax-50, d.ymin-50))
    array_pair_ref = gap_sweep << gap_sweep_array_pair
    array_pair_ref.move((-2700,400-1300*i))
    labelme(gap_sweep, array_pair_ref, f"Resonator gap = {resonator_gap*1e3:.0f} nm", text_spec=text_spec, anchor="NE", position=lambda d: (d.xmax-50, d.ymin-50))
my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43,deep_layer="DEEP_ETCH_EBL")
gc_test = gap_sweep << gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing)
gc_test.movex(origin=gc_test.xmin, destination=gap_sweep.xmin)
gc_test.movey(-5000)
for i in range(0,14,2):
    gf.routing.route_single(
        gap_sweep,
        port1=gc_test.ports[f'o{i+1}'],
        port2=gc_test.ports[f'o{i+2}'],
        cross_section=xs_routing,
    )
(layout_final << gap_sweep).move((-4000,480))
(layout_final << gap_sweep).move((1000,480))
(layout_final << gap_sweep).move((6000,480))
layout_final.show()

In [ ]:
gc_rib = partial(my_coupler,gc_spec,deep_layer="SHALLOW_ETCH")
gc_normal = partial(my_coupler,gc_spec,deep_layer="DEEP_ETCH_EBL")
xs_routing = cross_section_with_sleeves(0.43,5,radius=20, radius_min=15,sleeve_layer="DEEP_ETCH_EBL")
euler_test_spec = partial(euler_test, grating_coupler_spec=gc_normal,cross_section=xs_routing)
text_spec = partial(text_outline, size=50, outline_width=3, with_mask=False,layer='DEEP_ETCH_EBL')
def optical_test_array():
    c = gf.Component()
    ebta = c << euler_bend_test_array(euler_test_spec)
    ebta.move((0,0))
    labelme(c,ebta, text="bend_num:[5,10,15]", position=lambda c:(c.xmax+200, c.ymin+200), text_spec=text_spec,anchor='SW')
    cta = c << converter_test_array(partial(converter_test, grating_coupler_spec=gc_rib),pair_num_list=[10,20,50,60], deep_etch_layer="DEEP_ETCH_EBL")
    cta.move((0,-500))
    labelme(c,cta, text="pair_num:[10,20,50,60]", position="center", text_spec=text_spec,anchor='SW')
    st = spiral_test(grating_coupler_spec=gc_normal,cross_section=xs_routing)
    st_ref = c << st
    st_ref.move((0,-1100))
    labelme(c,st_ref, text=f"Spiral lengths:{st.info['Spiral lengths']}", position=lambda c:(c.xmax+200, c.ymin+500), text_spec=text_spec,anchor='SW')
    return c
(layout_final << optical_test_array()).move((3500-9600,-5000))
(layout_final << optical_test_array()).move((-2000,-5000))
def bend_circular_test_array():
    c = gf.Component()
    for i, radius in enumerate([5,10,20]):
        circular_bend_test_spec = partial(circular_bend_test, radius=radius, grating_coupler_spec=gc_normal, cross_section=xs)
        circular_bend_test_ref = c << euler_bend_test_array(circular_bend_test_spec)
        circular_bend_test_ref.move((0,-600*i))
        labelme(c,circular_bend_test_ref, text=f"Radius:{radius} um|bend_num:[5,10,15]", position=lambda c:(c.xmax+15, c.ymin+100), text_spec=text_spec,anchor='SW')
    return c
(layout_final << bend_circular_test_array()).move((2000,-5300))
(layout_final << bend_circular_test_array()).move((4600,-5300))
layout_final.show()

In [ ]:
etch_depth_arrays = [layout_final << etch_depth_array(layers=['DEEP_ETCH_EBL','SHALLOW_ETCH'],frame_layer='DEEP_ETCH_EBL') for _ in range(4)]
for i, eda in enumerate(etch_depth_arrays):
    eda.move(
        eda.center,
        (-8000 if i % 2 == 0 else 8000, -2500 if i // 2 == 0 else 2500),
    )

alignment_array = [
    layout_final << litho_caliper_array(
        types=["EBL","EBL","PL"], 
        layers=["DEEP_ETCH_EBL","SHALLOW_ETCH","MTOP"], 
        frame_layer='DEEP_ETCH_EBL') for _ in range(4)]
for i, aa in enumerate(alignment_array):
    aa.move(
        aa.center,
        (-8000 if i % 2 == 0 else 8000, -3500 if i // 2 == 0 else 3500),
    )

layout_final.show()